# Crawler del sitio Reclamos.cl

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from sklearn.utils import shuffle

In [7]:
def get_soup(page):
    url = f"https://www.reclamos.cl/?page={page}"
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
    return soup

In [3]:
s = get_soup(3)

In [8]:
def get_claims_urls(s):
    posts = s.find_all("td")
    #posts_urls = [post.find('a').get("href") for post in posts]
    posts_urls = list()
    for p in posts:
        if (p.find("a")) != None:
            posts_urls.append(p.find("a").get("href"))
    return posts_urls
def get_reqs(s):
    return [requests.get(url) for url in get_claims_urls(s)]

In [6]:
get_claims_urls(s)

['https://www.reclamos.cl/recsachile/reclamo/2019/oct/recsa_chile_acoso_telefonico',
 'https://www.reclamos.cl/reclamo/2019/oct/telas_chile_no_env_an_lo_pagado',
 'https://www.reclamos.cl/loteria/reclamo/2019/oct/loteria_deposite_2500_y_me_dice_que_no_ay_saldo_por_favor_resolver_asunto_saludos',
 'https://www.reclamos.cl/banco_bci/reclamo/2019/oct/bci_sucursal_la_cisterna_mala_gesti_n_reclamo_cuenta_cerrada_titular_fallecido',
 'https://www.reclamos.cl/hospital_salvador/reclamo/2019/oct/hospital_salvador_negligencia',
 'https://www.reclamos.cl/tottus/reclamo/2019/oct/tottus_no_se_entrega_de_producto_en_la_fecha_acordad',
 'https://www.reclamos.cl/loteria/reclamo/2019/oct/loteria_oidia_abone_en_loteria_10000_pesos_y_no_me_aparece_en_mi_pagina_de_kino_don',
 'https://www.reclamos.cl/inmbiliaria_lo_campino/reclamo/2019/oct/inmobiliaria_lo_campino_retraso_en_entrega_por_problemas_graves',
 'https://www.reclamos.cl/loteria/reclamo/2019/oct/loteria_no_se_abono_lo_que_se_cargo_de_mi_cuenta',


In [5]:
reqs = get_reqs(s)

KeyboardInterrupt: 

In [9]:
def crawl_req(req, verbose = False):
    s = BeautifulSoup(req.text)
    company = s.find("span",{"property":"v:itemreviewed"}).text
    subject = s.find("span",{"property":"v:summary"}).text
    date = s.find("span",{"property":"v:dtreviewed"}).get("content")
    description = s.find("span",{"property":"v:description"}).text.replace('\n','')
    
    if verbose == True:
        print("Company:",company)
        print("Subject:",subject)
        print("Date:",date)
        print("Description",description)
    return {"company":company,
           "subject":subject,
           "date":date,
           "description":description}

In [10]:
def crawl_reclamos(pages_range, filename = 'reclamos.csv', step=1):
    list_dic = []
    for i in range(pages_range[0],pages_range[1]+1,step):
        print(i)
        s = get_soup(i)
        reqs = get_reqs(s)
        for req in reqs:
            try:
                list_dic.append(crawl_req(req, False))
            except Exception as e:
                print(e)
    df = pd.DataFrame(list_dic)
    df.to_csv(filename)
    return df

In [11]:
df = crawl_reclamos([1,2])

1
2


In [17]:
df.head()

,company,subject,date,description
0,Wish,Robo/ fraude,2019-10-18,Compre en wish los primeros días de octubre de...
1,Siprt life,Cobro estácionamiento,2019-10-18,"Buenas tardes ,me llama la atención un gymasio..."
2,Correos de chile,No llega aún mi encomienda con n° de envío : c...,2019-10-18,Sres Correos de Chile:Con este N° de Envío ing...
3,Turbus,Mentiras,2019-10-18,Esta empresa por la.cual recordaba no viajar h...
4,Banco scotiabank,No pago de seguro cesantia,2019-10-18,El 30 de Agosto quede cesante por lo cual me a...


In [16]:
df.to_csv('sample_reclamos.csv')